#Önişleme

In [2]:
#Kütüphaneler yüklenir
import cv2 as cv
import os
import numpy as np
import shutil
from google.colab import drive

In [3]:
#Spectogramlar verilen linkten drive'a sabitlenir ve kişisel drive colab'a dosya olarak tanımlanır
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
#Veriler rar dosyasından çözümlenir
rar_file = 'gdrive/My Drive/spectrograms'
get_ipython().system_raw("unrar x rar_file")

In [5]:
#Path'ler oluşturulur
cwd = os.getcwd()
path = os.path.join(cwd, 'gdrive', 'MyDrive')
main_path = os.path.join(path,"spectrograms") #Ana path
saving_path = os.path.join(path,"Preprocess") #Önişlenmiş veriler için yeni path tanımlanır
if not os.path.exists(saving_path):
    os.makedirs(saving_path)

In [6]:
#Resizing için dimensionlar belirlenir
resize_x = 64
resize_y = 64
dim=(resize_x, resize_y)

In [ ]:
#Grayscale dönüşümü, resizing ve normalizasyon
for dir_name in os.listdir(main_path):
    label_path = os.path.join(main_path, dir_name)
    sub_dir_path = os.path.join(saving_path, dir_name)
    if not os.path.exists(sub_dir_path):
        os.makedirs(sub_dir_path)
    for spec in os.listdir(label_path):
        image_path = os.path.join(label_path, spec)
        image = cv.imread(image_path)
        gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #grayscaling
        resized_image = cv.resize(gray_image, dim, interpolation=cv.INTER_AREA) #resizing
        normalized_image = cv.normalize(resized_image, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX) #normalizing
        file_name = os.path.join(sub_dir_path, spec)
        cv.imwrite(file_name, normalized_image)

In [7]:
#Train, test ve validation setlerin ayrı label dosyalarında önişleme dosyasına kaydedilmesi  
for i in os.listdir(saving_path):
    os.makedirs(saving_path + '/train/' + i)
    os.makedirs(saving_path + '/test/' + i)
    os.makedirs(saving_path + '/validation/' + i)
    source = saving_path + '/' + i
    allFileNames = os.listdir(source)
    np.random.shuffle(allFileNames)

    train_ratio = 0.8
    test_ratio = 0.1
    validation_ratio = 0.1

    train_FileNames, test_FileNames, validation_FileNames = np.split(np.array(allFileNames),
                                               [int(len(allFileNames) * train_ratio),
                                                int(len(allFileNames) * (test_ratio + train_ratio))])

    train_FileNames = [source + '/' + name for name in train_FileNames.tolist()]
    test_FileNames = [source + '/' + name for name in test_FileNames.tolist()]
    validation_FileNames = [source + '/' + name for name in validation_FileNames.tolist()]

    for name in train_FileNames:
        shutil.copy(name, saving_path + '/train/' + i)

    for name in test_FileNames:
        shutil.copy(name, saving_path + '/test/' + i)

    for name in validation_FileNames:
        shutil.copy(name, saving_path + '/validation/' + i)